# Comparing the climatology against observation
#### Important: This needs to run with the "calc" python environment
- First import libraries

In [ ]:
import numpy as np
import netCDF4
import xarray as xr
import matplotlib.pyplot as plt

- Next, we need to read in the data. Let's pick dissolved oxygen (o2) from GFDL-ESM4
- World Ocean Atlas for observation

In [ ]:
! ls /global/cfs/cdirs/m3920/dataset/cmip6/GFDL-ESM4/o2*.nc > gfdlo2.txt
! cat gfdlo2.txt
! ls /global/cfs/cdirs/m3920/dataset/ocean/woa09/dissolved_oxy*annual*.nc > woao2.txt
! cat woao2.txt

- First read in the text file that contains the name of the file
- Use it to drive xarray.open_dataset
- See the graphic interface to explore metadata and attributes

In [ ]:
fid=open('gfdlo2.txt','r')
fn=fid.read().splitlines()
ds=xr.open_dataset(fn[0])
ds=ds.rename({'lev': 'depth',})
ds

In [ ]:
fid=open('woao2.txt','r')
fn=fid.read().splitlines()
dsobs=xr.open_dataset(fn[0],decode_times=False)
dsobs

- Calculate the annual mean climatology from 1950 to 2010
- To do so, we first select the correct time period and then apply .mean

In [ ]:
# put them on the same units
# model
unitconv0=1.0e6/1025. # from mol/m3 to umol/kg
o2_model = ds.o2.sel(time=slice("1950-01-01","2010-12-31")).mean(dim='time')*unitconv0
# obs
unitconv1=43.570      # from ml/l   to umol/kg
o2_obs = dsobs.o_an.isel(time=0)*unitconv1

- Calculate and plot the model-obs difference at a constant depth (100, 200, 400, 700m)

In [ ]:
# calculate the difference
o2_diff=o2_model - o2_obs
# colorbar options
colorbar_kwargs = {
    "orientation": "vertical",
    "label": "annual O2 bias from WOA 2009, micro-molO2/kg",
    "pad": 0.07,}
# plot 4 levels
o2_diff.sel(depth=[100, 200, 400, 700]).plot(col='depth',col_wrap=2,robust=True, 
                                             figsize=(7,5), cbar_kwargs=colorbar_kwargs)
#plt.savefig('o2bias_4panels.pdf',bbox_inches="tight")

- Next, we calculate the 0-700m vertically integrated O2. 
- Then make one plot that compares model and observation

In [ ]:
# first calculate cell area for 1x1 grid
X,Y=np.meshgrid(ds.lon,ds.lat)
r=6.37e6
areacell=(r*np.deg2rad(1.0))**2*np.cos(np.deg2rad(Y))

In [ ]:
# next calculate the vertical cell thickness 
z0=ds["depth"]
dZ=np.ediff1d(z0,to_begin=5)
# then volume element
volcell=np.empty((33,180,360))
for k in range(0,33):
    volcell[k,:,:]=areacell*dZ[k]
weight=xr.DataArray(data=volcell,dims=['depth','lat','lon'])

In [ ]:
# apply weights
K=(ds.depth>700)
weight[K,:,:]=0
# model
o2_model_weighted=o2_model.weighted(weight)
o2_model_col = o2_model_weighted.mean(dim='depth')
# obs
o2_obs_weighted=o2_obs.weighted(weight)
o2_obs_col = o2_obs_weighted.mean(dim='depth')

- Let's plot model - obs

In [ ]:
o2_diff_col = o2_model_col - o2_obs_col
o2_diff_col.plot(robust=True, cbar_kwargs=colorbar_kwargs)
plt.title('0-700m mean O2: model - obs')

- Let's calculate vertical profiles (global)

In [ ]:
# model
o2_model_weighted=o2_model.weighted(weight)
o2_model_prf = o2_model_weighted.mean(dim=['lat','lon'])
# obs
o2_obs_weighted=o2_obs.weighted(weight)
o2_obs_prf = o2_obs_weighted.mean(dim=['lat','lon'])

In [ ]:
plt.plot(o2_model_prf,ds.depth,label='model')
plt.plot(o2_obs_prf,ds.depth,label='woa 2009')
plt.legend()
plt.ylim(700,0)
plt.ylabel('depth')
plt.xlabel('O2 conc, micro-molO2/kg')
plt.title('global mean O2 profile: model - obs')